In [3]:
import scipy.io
import numpy as np 
import matplotlib.pyplot as plt
import pylab
from functools import reduce

In [4]:
w = ['O', 'N', 'A']
noise = list()
mat = open('training2017/REFERENCE-original.csv', 'r')
mat = list(map(lambda x: x.replace('\n', '').split(','), mat.readlines()))
for i in mat:
    if i[1] not in w:
        noise.append(int(i[0].replace('A', '')))
def num5(n):
    return '0' * (5 - len(str(n))) + str(n) # 5 -> 00005, A01234: 1234 -> 01234
def rank(arr):
    brr = [0] * len(arr)
    maxs = set()
    #topN = list()
    ler = 0
    while len(maxs) != len(arr):
        maximum = -10000
        I = 0
        for i in set(range(len(arr))) - maxs:
            if arr[i] > maximum:
                maximum = arr[i]
                I = i
        brr[I] = len(maxs)
        maxs.update({I})
    return brr

def series(data):#критерий серий
    ser = 1
    n1 = 0
    n2 = 0
    for i in range(1, len(data)):
        if data[i] * data[i - 1] <= 0:
            ser += 1
    for i in data:
        if i >= 0:
            n1 += 1
        else:
            n2 += 1
    E = 2 * n1 * n2/(n1 + n2) + 1
    Var = (2 * n1 * n2/(n1 + n2)**2) * (2 * n1 * n2 - n1 - n2)/(n1 + n2 - 1)
    return (ser - E)/np.sqrt(Var)

def otvet(data, x):#анализ на выбросы
    X = np.mean(data)
    V = np.var(data)
    count = 0
    for i in data:
        if i > X + x * np.sqrt(V) or i < X - x * np.sqrt(V):
            count += 1
    return count
#далее происходит отбор сначала по выбросам, потом из полученных по критерию серий
summ = 0
ryad = list()
flag = 1
it = 1
while flag:
    try:
        mat = scipy.io.loadmat('training2017/A' + num5(it) + '.mat')['val'][0]
        ryad.append([otvet(mat, 4.96), it])
        it += 1
    except:
        flag = 0
count = 0
ccount = 0
new = list()
for i in ryad:
    if i[0] <= 0.1:
        new.append(i)
dataset = list()
for i in new:
    mat = scipy.io.loadmat('training2017/A' + num5(i[1]) + '.mat')['val'][0]
    dataset.append([otvet(mat, 1.6), i[1]])
superdataset = list()
for i in dataset:
    if i[0] <= 1000:
        superdataset.append(i)
last = list()
for i in superdataset:
    mat = scipy.io.loadmat('training2017/A' + num5(i[1]) + '.mat')['val'][0]
    n = series(mat)
    if np.abs(n) < 14:
        last.append([n, i[1]])
numb = 0
for i in last:
    if i[1] in noise:
        numb += 1
print("Определены как шум: ", len(last))
print("Реальный шум: ", numb)

/Users/aplle/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: overflow encountered in short_scalars


Определены как шум:  26
Реальный шум:  7
